In [27]:
import numpy as np
import pandas as pd
import os
import datetime
import itertools
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import random
import tensorflow as tf
import math
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
import sys
from sklearn import preprocessing
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn import linear_model, tree, ensemble
from xgboost import plot_importance
import optuna
from sklearn.metrics import mean_squared_error as acc
import gc
from lightgbm import LGBMClassifier as lgb
from xgboost import XGBClassifier as clf
from scipy.stats import loguniform
from sklearn import linear_model, tree, ensemble
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier as cat
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import OrdinalEncoder

In [28]:
df=pd.read_csv('https://raw.githubusercontent.com/OmdenaAI/omdena-texas-homelessness/main/src/data/housing/2007-2020-PIT-Estimates-by-CoC.xlsx%20-%202020.csv?token=AT2ZEJMRVHQMPTLC3Z27DPTBKVK6Q')

In [29]:
df.head()

,CoC Number,CoC Name,CoC Category,"Overall Homeless, 2020","Overall Homeless - Under 18, 2020","Overall Homeless - Age 18 to 24, 2020","Overall Homeless - Over 24, 2020","Overall Homeless - Female, 2020","Overall Homeless - Male, 2020","Overall Homeless - Transgender, 2020","Overall Homeless - Gender Non-Conforming, 2020","Overall Homeless - Non-Hispanic/Non-Latino, 2020","Overall Homeless - Hispanic/Latino, 2020","Overall Homeless - White, 2020","Overall Homeless - Black or African American, 2020","Overall Homeless - Asian, 2020","Overall Homeless - American Indian or Alaska Native, 2020","Overall Homeless - Native Hawaiian or Other Pacific Islander, 2020","Overall Homeless - Multiple Races, 2020","Sheltered ES Homeless, 2020","Sheltered ES Homeless - Under 18, 2020","Sheltered ES Homeless - Age 18 to 24, 2020","Sheltered ES Homeless - Over 24, 2020","Sheltered ES Homeless - Female, 2020","Sheltered ES Homeless - Male, 2020","Sheltered ES Homeless - Transgender, 2020","Sheltered ES Homeless - Gender Non-Conforming, 2020","Sheltered ES Homeless - Non-Hispanic/Non-Latino, 2020","Sheltered ES Homeless - Hispanic/Latino, 2020","Sheltered ES Homeless - White, 2020","Sheltered ES Homeless - Black or African American, 2020","Sheltered ES Homeless - Asian, 2020","Sheltered ES Homeless - American Indian or Alaska Native, 2020","Sheltered ES Homeless - Native Hawaiian or Other Pacific Islander, 2020","Sheltered ES Homeless - Multiple Races, 2020","Sheltered TH Homeless, 2020","Sheltered TH Homeless - Under 18, 2020","Sheltered TH Homeless - Age 18 to 24, 2020","Sheltered TH Homeless - Over 24, 2020","Sheltered TH Homeless - Female, 2020","Sheltered TH Homeless - Male, 2020","Sheltered TH Homeless - Transgender, 2020","Sheltered TH Homeless - Gender Non-Conforming, 2020","Sheltered TH Homeless - Non-Hispanic/Non-Latino, 2020","Sheltered TH Homeless - Hispanic/Latino, 2020","Sheltered TH Homeless - White, 2020","Sheltered TH Homeless - Black or African American, 2020","Sheltered TH Homeless - Asian, 2020","Sheltered TH Homeless - American Indian or Alaska Native, 2020","Sheltered TH Homeless - Native Hawaiian or Other Pacific Islander, 2020","Sheltered TH Homeless - Multiple Races, 2020","Sheltered SH Homeless, 2020","Sheltered SH Homeless - Under 18, 2020","Sheltered SH Homeless - Age 18 to 24, 2020","Sheltered SH Homeless - Over 24, 2020","Sheltered SH Homeless - Female, 2020","Sheltered SH Homeless - Male, 2020","Sheltered SH Homeless - Transgender, 2020","Sheltered SH Homeless - Gender Non-Conforming, 2020","Sheltered SH Homeless - Non-Hispanic/Non-Latino, 2020","Sheltered SH Homeless - Hispanic/Latino, 2020","Sheltered SH Homeless - White, 2020","Sheltered SH Homeless - Black or African American, 2020","Sheltered SH Homeless - Asian, 2020","Sheltered SH Homeless - American Indian or Alaska Native, 2020","Sheltered SH Homeless - Native Hawaiian or Other Pacific Islander, 2020","Sheltered SH Homeless - Multiple Races, 2020","Sheltered Total Homeless, 2020","Sheltered Total Homeless - Under 18, 2020","Sheltered Total Homeless - Age 18 to 24, 2020","Sheltered Total Homeless - Over 24, 2020","Sheltered Total Homeless - Female, 2020","Sheltered Total Homeless - Male, 2020","Sheltered Total Homeless - Transgender, 2020","Sheltered Total Homeless - Gender Non-Conforming, 2020","Sheltered Total Homeless - Non-Hispanic/Non-Latino, 2020","Sheltered Total Homeless - Hispanic/Latino, 2020","Sheltered Total Homeless - White, 2020","Sheltered Total Homeless - Black or African American, 2020","Sheltered Total Homeless - Asian, 2020","Sheltered Total Homeless - American Indian or Alaska Native, 2020","Sheltered Total Homeless - Native Hawaiian or Other Pacific Islander, 2020","Sheltered Total Homeless - Multiple Races, 2020","Unsheltered Homeless, 2020","Unsheltered Homeless - Under 18, 2020","Unsheltered Homeless - Age 18 to 24, 2020","Unsheltered Homeless - Over 24, 2020","Unsheltered Homeless - Female, 2020","Unsheltered Homeless - Male, 2020","U

In [30]:
df.describe()

,"Sheltered ES Homeless - Transgender, 2020","Sheltered ES Homeless - Gender Non-Conforming, 2020","Sheltered TH Homeless - Transgender, 2020","Sheltered TH Homeless - Gender Non-Conforming, 2020","Sheltered TH Homeless - Asian, 2020","Sheltered TH Homeless - Native Hawaiian or Other Pacific Islander, 2020","Sheltered SH Homeless - Under 18, 2020","Sheltered SH Homeless - Age 18 to 24, 2020","Sheltered SH Homeless - Female, 2020","Sheltered SH Homeless - Transgender, 2020","Sheltered SH Homeless - Gender Non-Conforming, 2020","Sheltered SH Homeless - Hispanic/Latino, 2020","Sheltered SH Homeless - White, 2020","Sheltered SH Homeless - Black or African American, 2020","Sheltered SH Homeless - Asian, 2020","Sheltered SH Homeless - American Indian or Alaska Native, 2020","Sheltered SH Homeless - Native Hawaiian or Other Pacific Islander, 2020","Sheltered SH Homeless - Multiple Races, 2020","Sheltered Total Homeless - Gender Non-Conforming, 2020","Sheltered ES Homeless Individuals - Transgender, 2020","Sheltered ES Homeless Individuals - Gender Non-Conforming, 2020","Sheltered TH Homeless Individuals - Under 18, 2020","Sheltered TH Homeless Individuals - Transgender, 2020","Sheltered TH Homeless Individuals - Gender Non-Conforming, 2020","Sheltered TH Homeless Individuals - Asian, 2020","Sheltered TH Homeless Individuals - American Indian or Alaska Native, 2020","Sheltered TH Homeless Individuals - Native Hawaiian or Other Pacific Islander, 2020","Sheltered SH Homeless Individuals - Under 18, 2020","Sheltered SH Homeless Individuals - Age 18 to 24, 2020","Sheltered SH Homeless Individuals - Female, 2020","Sheltered SH Homeless Individuals - Transgender, 2020","Sheltered SH Homeless Individuals - Gender Non-Conforming, 2020","Sheltered SH Homeless Individuals - Hispanic/Latino, 2020","Sheltered SH Homeless Individuals - White, 2020","Sheltered SH Homeless Individuals - Black or African American, 2020","Sheltered SH Homeless Individuals - Asian, 2020","Sheltered SH Homeless Individuals - American Indian or Alaska Native, 2020","Sheltered SH Homeless Individuals - Native Hawaiian or Other Pacific Islander, 2020","Sheltered SH Homeless Individuals - Multiple Races, 2020","Sheltered Total Homeless Individuals - Gender Non-Conforming, 2020","Unsheltered Homeless Individuals - Gender Non-Conforming, 2020","Overall Homeless People in Families - Transgender, 2020","Overall Homeless People in Families - Gender Non-Conforming, 2020","Sheltered ES Homeless People in Families - Transgender, 2020","Sheltered ES Homeless People in Families - Gender Non-Conforming, 2020","Sheltered TH Homeless People in Families - Transgender, 2020","Sheltered TH Homeless People in Families - Gender Non-Conforming, 2020","Sheltered TH Homeless People in Families - Asian, 2020","Sheltered TH Homeless People in Families - American Indian or Alaska Native, 2020","Sheltered TH Homeless People in Families - Native Hawaiian or Other Pacific Islander, 2020","Sheltered Total Homeless People in Families - Transgender, 2020","Sheltered Total Homeless People in Families - Gender Non-Conforming, 2020","Unsheltered Homeless People in Families - Transgender, 2020","Unsheltered Homeless People in Families - Gender Non-Conforming, 2020","Unsheltered Homeless People in Families - Asian, 2020","Unsheltered Homeless People in Families - American Indian or Alaska Native, 2020","Sheltered SH Chronically Homeless, 2020","Sheltered SH Chronically Homeless Individuals, 2020","Overall Homeless Veterans - Transgender, 2020","Overall Homeless Veterans - Gender Non-Conforming, 2020","Overall Homeless Veterans - Asian, 2020","Overall Homeless Veterans - Native Hawaiian or Other Pacific Islander, 2020","Sheltered ES Homeless Veterans - Female, 2020","Sheltered ES Homeless Veterans - Transgender, 2020","Sheltered ES Homeless Veterans - Gender Non-Conforming, 2020","Sheltered ES Homeless Veterans - Hispanic/Latino, 2020","Sheltered ES Homeless Veterans - Asian, 2020","Sheltered ES Homeless Vete

In [31]:
df.isnull().sum().sort_values(ascending = False)

CoC Category                                                                     3
CoC Number                                                                       2
Unsheltered Homeless Veterans - Hispanic/Latino, 2020                            2
Unsheltered Homeless Veterans, 2020                                              2
Unsheltered Homeless Veterans - Female, 2020                                     2
                                                                                ..
Sheltered Total Homeless Individuals - American Indian or Alaska Native, 2020    2
Sheltered Total Homeless Individuals - Asian, 2020                               2
Sheltered Total Homeless Individuals - Black or African American, 2020           2
Sheltered Total Homeless Individuals - White, 2020                               2
Unsheltered Homeless Children of Parenting Youth, 2020                           2
Length: 542, dtype: int64

In [32]:
df.isnull().sum(axis=1).sort_values(ascending = False).head(15)

393    542
394    541
392      2
259      0
269      0
268      0
267      0
266      0
265      0
264      0
263      0
262      0
261      0
260      0
258      0
dtype: int64

In [33]:
df.iloc[391:395,:]

,CoC Number,CoC Name,CoC Category,"Overall Homeless, 2020","Overall Homeless - Under 18, 2020","Overall Homeless - Age 18 to 24, 2020","Overall Homeless - Over 24, 2020","Overall Homeless - Female, 2020","Overall Homeless - Male, 2020","Overall Homeless - Transgender, 2020","Overall Homeless - Gender Non-Conforming, 2020","Overall Homeless - Non-Hispanic/Non-Latino, 2020","Overall Homeless - Hispanic/Latino, 2020","Overall Homeless - White, 2020","Overall Homeless - Black or African American, 2020","Overall Homeless - Asian, 2020","Overall Homeless - American Indian or Alaska Native, 2020","Overall Homeless - Native Hawaiian or Other Pacific Islander, 2020","Overall Homeless - Multiple Races, 2020","Sheltered ES Homeless, 2020","Sheltered ES Homeless - Under 18, 2020","Sheltered ES Homeless - Age 18 to 24, 2020","Sheltered ES Homeless - Over 24, 2020","Sheltered ES Homeless - Female, 2020","Sheltered ES Homeless - Male, 2020","Sheltered ES Homeless - Transgender, 2020","Sheltered ES Homeless - Gender Non-Conforming, 2020","Sheltered ES Homeless - Non-Hispanic/Non-Latino, 2020","Sheltered ES Homeless - Hispanic/Latino, 2020","Sheltered ES Homeless - White, 2020","Sheltered ES Homeless - Black or African American, 2020","Sheltered ES Homeless - Asian, 2020","Sheltered ES Homeless - American Indian or Alaska Native, 2020","Sheltered ES Homeless - Native Hawaiian or Other Pacific Islander, 2020","Sheltered ES Homeless - Multiple Races, 2020","Sheltered TH Homeless, 2020","Sheltered TH Homeless - Under 18, 2020","Sheltered TH Homeless - Age 18 to 24, 2020","Sheltered TH Homeless - Over 24, 2020","Sheltered TH Homeless - Female, 2020","Sheltered TH Homeless - Male, 2020","Sheltered TH Homeless - Transgender, 2020","Sheltered TH Homeless - Gender Non-Conforming, 2020","Sheltered TH Homeless - Non-Hispanic/Non-Latino, 2020","Sheltered TH Homeless - Hispanic/Latino, 2020","Sheltered TH Homeless - White, 2020","Sheltered TH Homeless - Black or African American, 2020","Sheltered TH Homeless - Asian, 2020","Sheltered TH Homeless - American Indian or Alaska Native, 2020","Sheltered TH Homeless - Native Hawaiian or Other Pacific Islander, 2020","Sheltered TH Homeless - Multiple Races, 2020","Sheltered SH Homeless, 2020","Sheltered SH Homeless - Under 18, 2020","Sheltered SH Homeless - Age 18 to 24, 2020","Sheltered SH Homeless - Over 24, 2020","Sheltered SH Homeless - Female, 2020","Sheltered SH Homeless - Male, 2020","Sheltered SH Homeless - Transgender, 2020","Sheltered SH Homeless - Gender Non-Conforming, 2020","Sheltered SH Homeless - Non-Hispanic/Non-Latino, 2020","Sheltered SH Homeless - Hispanic/Latino, 2020","Sheltered SH Homeless - White, 2020","Sheltered SH Homeless - Black or African American, 2020","Sheltered SH Homeless - Asian, 2020","Sheltered SH Homeless - American Indian or Alaska Native, 2020","Sheltered SH Homeless - Native Hawaiian or Other Pacific Islander, 2020","Sheltered SH Homeless - Multiple Races, 2020","Sheltered Total Homeless, 2020","Sheltered Total Homeless - Under 18, 2020","Sheltered Total Homeless - Age 18 to 24, 2020","Sheltered Total Homeless - Over 24, 2020","Sheltered Total Homeless - Female, 2020","Sheltered Total Homeless - Male, 2020","Sheltered Total Homeless - Transgender, 2020","Sheltered Total Homeless - Gender Non-Conforming, 2020","Sheltered Total Homeless - Non-Hispanic/Non-Latino, 2020","Sheltered Total Homeless - Hispanic/Latino, 2020","Sheltered Total Homeless - White, 2020","Sheltered Total Homeless - Black or African American, 2020","Sheltered Total Homeless - Asian, 2020","Sheltered Total Homeless - American Indian or Alaska Native, 2020","Sheltered Total Homeless - Native Hawaiian or Other Pacific Islander, 2020","Sheltered Total Homeless - Multiple Races, 2020","Unsheltered Homeless, 2020","Unsheltered Homeless - Under 18, 2020","Unsheltered Homeless - Age 18 to 24, 2020","Unsheltered Homeless - Over 24, 2020","Unsheltered Homeless - Female, 2020","Unsheltered Homeless - Male, 2020","U

In [34]:
df=df.dropna()
df=df.reset_index(drop=True)

In [35]:
categorical_cols = df.select_dtypes('object').columns

In [36]:
for i in categorical_cols[3:]:
    df[i]=df[i].apply(lambda x: x.replace(",", ""))

In [37]:
for i in categorical_cols[3:]:
    df[i]=df[i].astype(float)

In [38]:
df['CoC Category'].unique()

array(['Other Largely Urban CoC', 'Largely Rural CoC',
       'Largely Suburban CoC', 'Major City CoC'], dtype=object)

In [39]:
enc=OrdinalEncoder()
df[['CoC Category']]=enc.fit_transform(df[['CoC Category']])

In [40]:
df['CoC Number'].unique()

array(['AK-500', 'AK-501', 'AL-500', 'AL-501', 'AL-502', 'AL-503',
       'AL-504', 'AL-505', 'AL-506', 'AL-507', 'AR-500', 'AR-501',
       'AR-503', 'AR-505', 'AZ-500', 'AZ-501', 'AZ-502', 'CA-500',
       'CA-501', 'CA-502', 'CA-503', 'CA-504', 'CA-505', 'CA-506',
       'CA-507', 'CA-508', 'CA-509', 'CA-510', 'CA-511', 'CA-512',
       'CA-513', 'CA-514', 'CA-515', 'CA-516', 'CA-517', 'CA-518',
       'CA-519', 'CA-520', 'CA-521', 'CA-522', 'CA-523', 'CA-524',
       'CA-525', 'CA-526', 'CA-527', 'CA-529', 'CA-530', 'CA-531',
       'CA-600', 'CA-601', 'CA-602', 'CA-603', 'CA-604', 'CA-606',
       'CA-607', 'CA-608', 'CA-609', 'CA-611', 'CA-612', 'CA-613',
       'CA-614', 'CO-500', 'CO-503', 'CO-504', 'CO-505', 'CT-503',
       'CT-505', 'DC-500', 'DE-500', 'FL-500', 'FL-501', 'FL-502',
       'FL-503', 'FL-504', 'FL-505', 'FL-506', 'FL-507', 'FL-508',
       'FL-509', 'FL-510', 'FL-511', 'FL-512', 'FL-513', 'FL-514',
       'FL-515', 'FL-517', 'FL-518', 'FL-519', 'FL-520', 'FL-6

In [41]:
#Ak is for Alaska , Az for arizona, and similar

In [42]:
df['CoC Number']=df['CoC Number'].apply(lambda x: x.split('-')[0])

In [43]:
df['CoC Number'].unique()

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD',
       'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC',
       'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'],
      dtype=object)

In [44]:
df[['CoC Number']]=enc.fit_transform(df[['CoC Number']])

In [45]:
df['CoC Name'].unique()

array(['Anchorage CoC', 'Alaska Balance of State CoC',
       'Birmingham/Jefferson, St. Clair, Shelby Counties CoC',
       'Mobile City & County/Baldwin County CoC',
       'Florence/Northwest Alabama CoC', 'Huntsville/North Alabama CoC',
       'Montgomery City & County CoC', 'Gadsden/Northeast Alabama CoC',
       'Tuscaloosa City & County CoC', 'Alabama Balance of State CoC',
       'Little Rock/Central Arkansas CoC',
       'Fayetteville/Northwest Arkansas CoC',
       'Arkansas  Balance of State  CoC', 'Southeast Arkansas CoC',
       'Arizona Balance of State CoC', 'Tucson/Pima County CoC',
       'Phoenix, Mesa/Maricopa County CoC',
       'San Jose/Santa Clara City & County CoC', 'San Francisco CoC',
       'Oakland, Berkeley/Alameda County CoC',
       'Sacramento City & County CoC',
       'Santa Rosa, Petaluma/Sonoma County CoC',
       'Richmond/Contra Costa County CoC',
       'Salinas/Monterey, San Benito Counties CoC', 'Marin County CoC',
       'Watsonville/Santa Cruz

In [46]:
#all are unique in Coc Name and no relation among them so droping this column is a good option
df=df.drop('CoC Name', axis=1)

In [47]:
df.to_csv('2007-2020-PIT-Estimates-by-CoC.csv', index=False)